In [1]:
import jax
from jax import random, jit, value_and_grad, nn
import jax.numpy as jnp
import numpy as np
import haiku as hk
import optax
from typing import Mapping, Any, Tuple, List, Iterator, Optional, Dict
from jax.typing import ArrayLike
from meta_transformer import utils, preprocessing, torch_utils, module_path
from meta_transformer.meta_model import create_meta_model
from meta_transformer.meta_model import MetaModelConfig as ModelConfig
import wandb
import os
import argparse
from dataclasses import asdict
from meta_transformer.train import Updater, Logger
from meta_transformer.data import data_iterator, split_data
from augmentations import permute_checkpoint
#permute_checkpoint = lambda *args, **kwargs: [None]

import json

In [4]:
DATA_DIR = os.path.join(module_path, 'data/david_backdoors/mnist/models')
#INPUTS_DIRNAME = "poison_easy"  # for CIFAR-10
INPUTS_DIRNAME = "poison"  # for MNIST
TARGETS_DIRNAME = "clean"
architecture = torch_utils.CNNSmall()  # for MNIST

# Load model checkpoints
print("Loading data...")
inputs, targets, get_pytorch_model = torch_utils.load_input_and_target_weights(
    model = architecture,
    num_models=10, 
    data_dir=DATA_DIR,
    inputs_dirname=INPUTS_DIRNAME,
)

NameError: name 'module_path' is not defined

In [5]:
model = get_pytorch_model(inputs[0])

NameError: name 'get_pytorch_model' is not defined

In [ ]:
# plan:
# 1. save model after train
# 2. load model in validation script, generate a ton of reconstructed models
# 3. pytorch val script: load reconstructed models, evaluate on poisoned and clean data

# thingy

In [37]:
# pytorch script to compute metrics for pytorch model on MNIST / CIFAR-10

import torch
import torch.nn as nn
import os
from typing import Dict, Tuple
import numpy as np
import datasets


def get_accuracy(model: nn.Module, inputs: np.ndarray, targets: np.ndarray) -> float:
    """Compute accuracy of model on inputs and targets."""
    with torch.no_grad():
        outputs = model(torch.from_numpy(inputs).float())
        _, predicted = torch.max(outputs.data, 1)
        total = targets.size
        correct = (predicted == torch.from_numpy(targets)).sum().item()
        return correct / total
    

def get_loss(model: nn.Module, inputs: np.ndarray, targets: np.ndarray) -> float:
    """Compute loss of model on inputs and targets."""
    with torch.no_grad():
        outputs = model(torch.from_numpy(inputs).float())
        loss = nn.CrossEntropyLoss()(outputs, torch.from_numpy(targets))
        return loss.item()


def load_mnist_test_set():
    dataset = datasets.load_dataset('mnist')
    dataset = dataset.with_format("torch")

    # Split the dataset into train and test sets
    test_data, test_labels = dataset['test']["image"], dataset['test']["label"]
    test_data  = test_data.reshape(-1, 1, 28, 28) / 255.

    test_data, test_labels = test_data.to('cuda'), test_labels.to('cuda')
    return TensorDataset(test_data, test_labels)

Found cached dataset mnist (/home/lauro/.cache/huggingface/datasets/mnist/mnist/1.0.0/9d494b7f466d6931c64fb39d58bb1249a4d85c9eb9865d9bc20960b999e2a332)
100%|██████████| 2/2 [00:00<00:00, 943.60it/s]


In [63]:
from gen_models import poison
from gen_models import config
cfg = config.Config()

In [64]:
# prepare data on GPU
train_clean = TensorDataset(test_data.to('cuda'), test_labels.to('cuda'))
train_poison = poison.poison_set(dd, train=False, cfg=cfg)

In [67]:
len(train_poison.tensors[1])

10000

In [74]:
train_clean.tensors[0].sum().shape

torch.Size([])